<a href="https://colab.research.google.com/github/rexian/ML/blob/main/langsmith/rag/evaluation/deepeval_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q h5py typing-extensions wheel
!pip install -U -q chromadb tiktoken pypdf
!pip install -U -q langchain-community langchain_huggingface

In [ ]:
!pip3 install -U -q deepeval aiobotocore

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [12]:
!pip install -U -q faiss-cpu 	langchain-nvidia-ai-endpoints

In [16]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(
    "./spaul/Deep-Learning-with-PyTorch.pdf",
)
document = loader.load()

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)
text = text_splitter.split_documents(document)

In [28]:
from langchain.vectorstores import Chroma
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=text,
                                 collection_name="deepeval",
                                 embedding=hf,
                                 persist_directory=persist_directory)

In [29]:
vectordb.persist()
vectordb = None

In [30]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=hf)

In [22]:
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents("What is natural language processing?")

<ipython-input-22-533c3e6fbc68>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("What is natural language processing?")


In [25]:
import os
import getpass
os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA AI Endpoints: ")

Enter API key for NVIDIA AI Endpoints: ··········


In [27]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA

template = """

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say 'Ah snap homie, I ain't gonna front. I don't know.`, don't try to make up an answer.
Use three sentences maximum, relevant analogies, and keep the answer as concise as possible.
Use the active voice, and speak directly to the reader using concise language.
{context}
Question: {question}
Helpful Answer:

"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

nvidia_model = ChatNVIDIA(model="meta/llama3-70b-instruct")

qa_chain = RetrievalQA.from_chain_type(
    nvidia_model,
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

query = "What is deep learning?"
result = qa_chain.invoke({"query": query})

result["result"]

"Deep learning is a type of machine learning that deals with training mathematical entities named deep neural networks on the basis of examples, allowing them to approximate highly nonlinear phenomena and capture the inherent structure of data. It's like a super-smart, data-driven approach to solving problems, where you don't always need to understand the underlying phenomenon, but can still get accurate results. Think of it like a submarine that can swim, without needing to understand the intricacies of swimming!"

In [57]:
import chromadb
persistent_client = chromadb.PersistentClient(path="db")
collection = persistent_client.get_or_create_collection("deepeval")
collection.name


'deepeval'

In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA

def search(query):
  query_embedding = hf.embed_query(query)

  res = collection.query(
    query_embeddings=[query_embedding],
    #query_texts=[query],
    n_results=3,  # Retrieve top-K matches
    include=["documents"]
  )
  result = []
  for doc in res['documents']:
    result.append(doc[0])
  return  result if result else None

query = "What is tensor in deep learning?"
retrieval_context = search(query)
print(retrieval_context)

llm = ChatNVIDIA(model="meta/llama3-70b-instruct")

prompt_template = """
Answer the user question based on the supporting context.

User Question:
{input}

Supporting Context:
{retrieval_context}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

chain = prompt | llm

actual_output = chain.invoke({"input": query, "retrieval_context": retrieval_context})
print(actual_output.content)
expected_output = "a tensor in deep learning is a fundamental data structure that is a generalization of vectors and matrices to an arbitrary number of dimensions"



['17\nintermediate representations, and as output. This chapter is devoted to providing pre -\ncisely that understanding.\n T o  t h i s  e n d ,  P y T o r c h  i n t r o d u c e s  a  f u n dam ental data structure: the tensor. For  \nthose who come from mathematics, ph ysics, or engineering, the term tensor  c o m e s \nbundled with the notion of spaces, refe rence systems, and transformations between  \nthem. For everyone else, tensor  refers to the generalization  of vectors and matrices to  \nan arbitrary number of dimensions, as sh own in figure 2.2. Another name for the  \nsame concept is multidimensional arrays . The dimensionality of a tensor coincides with  \nthe number of indexes used to refer to scalar values within the tensor. \nFigure 2.2 Tensors are the building blocks for representing data in PyTorch\n PyTorch isn’t not the only library that deals with multidimen sional arrays. NumPy  \nis by far the most popular mu ltidimensional-array library, to the point that it ha

In [75]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input="What is tensor in deep learning?",
    actual_output = actual_output,
    expected_output = expected_output,
    retrieval_context = retrieval_context
)

In [86]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OPEN AI: ")

Enter API key for OPEN AI: ··········


### Evaluating Retrieval

In [ ]:
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)

contextual_precision = ContextualPrecisionMetric()
contextual_recall = ContextualRecallMetric()
contextual_relevancy = ContextualRelevancyMetric()

contextual_precision.measure(test_case)
print("Score: ", contextual_precision.score)
print("Reason: ", contextual_precision.reason)

contextual_recall.measure(test_case)
print("Score: ", contextual_recall.score)
print("Reason: ", contextual_recall.reason)

contextual_relevancy.measure(test_case)
print("Score: ", contextual_relevancy.score)
print("Reason: ", contextual_relevancy.reason)

### Evaluating Generation

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric

answer_relevancy = AnswerRelevancyMetric()
faithfulness = FaithfulnessMetric()

answer_relevancy.measure(test_case)
print("Score: ", answer_relevancy.score)
print("Reason: ", answer_relevancy.reason)

faithfulness.measure(test_case)
print("Score: ", faithfulness.score)
print("Reason: ", faithfulness.reason)